# Public Transit Play II 

1. Find "The shortest route to iterate all subway lines in ShangHai."
2. Find "The Longest route without same edge in subway network of ShangHai & Beijing"

Answers of 1. can be found on Zhihu: https://www.zhihu.com/question/629922852/answer/54798222956


In [45]:
import gurobipy as grb

In [65]:
file_path = "./data/PublicTransit/SH_new.txt"  # 替换为你的实际文件路径
# file_path = "./data/PublicTransit/BJ.txt"  # 替换为你的实际文件路径


In [66]:
import gurobipy as grb
def read_metro_data(file_path):
    from collections import defaultdict
    # 邻接表表示的地铁网络图
    metro_network = defaultdict(dict)
    line_routes = dict()
    station_name_to_idx = dict()
    idx_to_station_name = dict()
    cnt_idx = 1
    # transship = dict()
    with open(file_path, 'r', encoding='utf-8') as file:
        current_line = None
        for line in file:
            line = line.strip()
            if line.startswith("Line"):
                # 读取线路名
                current_line = line
                line_routes[line] = list()
            elif line:
                # 读取站点和距离信息
                station1, station2, distance = line.split(",")
                if station1 not in station_name_to_idx:
                    station_name_to_idx[station1] = cnt_idx
                    idx_to_station_name[cnt_idx] = station1
                    cnt_idx += 1
                if station2 not in station_name_to_idx:
                    station_name_to_idx[station2] = cnt_idx
                    idx_to_station_name[cnt_idx] = station2
                    cnt_idx += 1
                distance = int(distance)  # 转换距离为整数
                metro_network[station1][station2] = distance
                metro_network[station2][station1] = distance
                # metro_network[station1][station2] = 1
                # metro_network[station2][station1] = 1
                line_routes[current_line].append(f"{station1},{station2}")
                

    return metro_network, line_routes, station_name_to_idx, idx_to_station_name

# 示例调用
metro_network, line_routes, station_name_to_idx, idx_to_station_name = read_metro_data(file_path)

# 打印部分结果查看结构
for station, connections in list(metro_network.items())[:5]:
    print(f"Station: {station}, Connections: {connections}")

Station: 莘庄, Connections: {'外环路': 1303, '春申路': 1588}
Station: 外环路, Connections: {'莘庄': 1303, '莲花路': 1459}
Station: 莲花路, Connections: {'外环路': 1459, '锦江乐园': 1633}
Station: 锦江乐园, Connections: {'莲花路': 1633, '上海南站': 2089}
Station: 上海南站, Connections: {'锦江乐园': 2089, '漕宝路': 1661, '石龙路': 1314, '华东理工大学': 1623, '桂林公园': 1828}


### Gurobi Version (Fast!): Q1

In [69]:
ptn = grb.Model("PTN")
idx_to_station_name[0] = "Dummy"
station_name_to_idx["Dummy"] = 0
x = {}
y = {}
n = {}
N = len(station_name_to_idx.keys())
M = 2000
out_flow = dict()

for i in range(1, N):
    x[0, i] = ptn.addVar(vtype=grb.GRB.BINARY,obj = 0, name=f"x_Dummy_{idx_to_station_name[i]}")
    x[i, 0] = ptn.addVar(vtype=grb.GRB.BINARY,obj = 0, name=f"x_{idx_to_station_name[i]}_Dummy")
    y[i] = ptn.addVar(vtype=grb.GRB.BINARY,obj = 0, name=f"y_{idx_to_station_name[i]}")
    n[i] = ptn.addVar(vtype=grb.GRB.INTEGER,obj = 0, lb = 0, name=f"n_{idx_to_station_name[i]}")
    out_flow[i] = []

for station, connections in list(metro_network.items()):
    idx_1 = station_name_to_idx[station]
    for next_station, dist in connections.items():
        idx_2 = station_name_to_idx[next_station]
        x[idx_1, idx_2] = ptn.addVar(vtype=grb.GRB.BINARY,obj = dist, name=f"x_{station}_{next_station}")
        # x[idx_1, idx_2] = ptn.addVar(vtype=grb.GRB.BINARY,obj = 1, name=f"x_{station}_{next_station}")
        out_flow[idx_1].append(idx_2)
# 针对换乘的情况, 给出带转运的解


# 流出的
for k, v in out_flow.items():

    v_new = v + [0]
    ptn.addConstr(grb.quicksum(x[k, j] for j in v_new) == y[k], name=f"out_flow_{idx_to_station_name[k]}")
    ptn.addConstr(grb.quicksum(x[i, k] for i in v_new) == y[k], name=f"in_flow_{idx_to_station_name[k]}")


ptn.addConstr(grb.quicksum(x[i, 0] for i in range(1, N)) == 1, name=f"Dummy_balance1")
ptn.addConstr(grb.quicksum(x[0, i] for i in range(1, N)) == 1, name=f"Dummy_balance2")

for i in range(1, N):
    ptn.addConstr(n[i] <= M * y[i], name = f"N_{i}")

for station, connections in list(metro_network.items()):
    idx_1 = station_name_to_idx[station]
    for next_station, dist in connections.items():
        idx_2 = station_name_to_idx[next_station]
        ptn.addConstr(n[idx_2] - n[idx_1] >= 1 + M * (x[idx_1, idx_2] - 1), name = f"n_{idx_1}_{idx_2}")

# 限制每条线都至少要走一次
for k, edges in line_routes.items():
    left_edges = []
    for edge in edges:
        o_, d_ = edge.split(",")
        o_idx, d_idx = station_name_to_idx[o_], station_name_to_idx[d_]
        left_edges.append([o_idx, d_idx])
        left_edges.append([d_idx, o_idx])
    ptn.addConstr(grb.quicksum(x[i, j] for i, j in left_edges) >= 1, name = f"line_{k}")

# 处理 Line 3 + Line 4 的情况(上海地铁)
edges_3, edges_4 = line_routes["Line 3"], line_routes["Line 4"]
new_edges = list(set(edges_3 + edges_4))
left_edges = []
for edge in new_edges:
    o_, d_ = edge.split(",")
    o_idx, d_idx = station_name_to_idx[o_], station_name_to_idx[d_]
    left_edges.append([o_idx, d_idx])
    left_edges.append([d_idx, o_idx])
ptn.addConstr(grb.quicksum(x[i, j] for i, j in left_edges) >= 2, name = f"spec_line_3/4")

edges_2, edges_10 = line_routes["Line 2"], line_routes["Line 10"]
new_edges = list(set(edges_2 + edges_10))
left_edges = []
for edge in new_edges:
    o_, d_ = edge.split(",")
    o_idx, d_idx = station_name_to_idx[o_], station_name_to_idx[d_]
    left_edges.append([o_idx, d_idx])
    left_edges.append([d_idx, o_idx])
ptn.addConstr(grb.quicksum(x[i, j] for i, j in left_edges) >= 2, name = f"spec_line_10/2")


ptn.modelSense = grb.GRB.MINIMIZE
# ptn.modelSense = grb.GRB.MAXIMIZE
result_routes = []
ptn.optimize()
if (ptn.status == grb.GRB.status.OPTIMAL):
    solType = 'IP_Optimal'
    ofv = ptn.getObjective().getValue()
    print(f"BEST VALUE:{ofv}")
    for i, j in x:
        if (x[i, j].x > 0.5):
            route = f"{idx_to_station_name[i]},{idx_to_station_name[j]}"
            result_routes.append(route)
            print(route)

    gap = 0
    lb = ofv
    ub = ofv
    runtime = ptn.Runtime

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M3 Pro
Thread count: 11 physical cores, 11 logical processors, using up to 11 threads

Optimize a model with 2208 rows, 2592 columns and 9296 nonzeros
Model fingerprint: 0x701983a3
Variable types: 0 continuous, 2592 integer (2186 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [6e+02, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Presolve time: 0.01s
Presolved: 2208 rows, 2592 columns, 9296 nonzeros
Variable types: 0 continuous, 2592 integer (2186 binary)

Root relaxation: objective 1.541800e+04, 211 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 15418.0000    0   56          - 15418.0000      -     -    0s
     0     0 16532.0000    0   81          - 16532.0000 

## Gurobi for Q2

In [71]:
ptn = grb.Model("PTN")
idx_to_station_name[0] = "Dummy"
station_name_to_idx["Dummy"] = 0
x = {}
y = {}
n = {}
N = len(station_name_to_idx.keys())
M = 2000
out_flow = dict()

for i in range(1, N):
    x[0, i] = ptn.addVar(vtype=grb.GRB.BINARY,obj = 0, name=f"x_Dummy_{idx_to_station_name[i]}")
    x[i, 0] = ptn.addVar(vtype=grb.GRB.BINARY,obj = 0, name=f"x_{idx_to_station_name[i]}_Dummy")
    y[i] = ptn.addVar(vtype=grb.GRB.BINARY,obj = 0, name=f"y_{idx_to_station_name[i]}")
    n[i] = ptn.addVar(vtype=grb.GRB.INTEGER,obj = 0, lb = 0, name=f"n_{idx_to_station_name[i]}")
    out_flow[i] = []

for station, connections in list(metro_network.items()):
    idx_1 = station_name_to_idx[station]
    for next_station, dist in connections.items():
        idx_2 = station_name_to_idx[next_station]
        x[idx_1, idx_2] = ptn.addVar(vtype=grb.GRB.BINARY,obj = dist, name=f"x_{station}_{next_station}")
        # x[idx_1, idx_2] = ptn.addVar(vtype=grb.GRB.BINARY,obj = 1, name=f"x_{station}_{next_station}")
        out_flow[idx_1].append(idx_2)
# 针对换乘的情况, 给出带转运的解


# 流出的
for k, v in out_flow.items():

    v_new = v + [0]
    ptn.addConstr(grb.quicksum(x[k, j] for j in v_new) == y[k], name=f"out_flow_{idx_to_station_name[k]}")
    ptn.addConstr(grb.quicksum(x[i, k] for i in v_new) == y[k], name=f"in_flow_{idx_to_station_name[k]}")


ptn.addConstr(grb.quicksum(x[i, 0] for i in range(1, N)) == 1, name=f"Dummy_balance1")
ptn.addConstr(grb.quicksum(x[0, i] for i in range(1, N)) == 1, name=f"Dummy_balance2")

for i in range(1, N):
    ptn.addConstr(n[i] <= M * y[i], name = f"N_{i}")

for station, connections in list(metro_network.items()):
    idx_1 = station_name_to_idx[station]
    for next_station, dist in connections.items():
        idx_2 = station_name_to_idx[next_station]
        ptn.addConstr(n[idx_2] - n[idx_1] >= 1 + M * (x[idx_1, idx_2] - 1), name = f"n_{idx_1}_{idx_2}")

# 限制每条线都至少要走一次
# for k, edges in line_routes.items():
#     left_edges = []
#     for edge in edges:
#         o_, d_ = edge.split(",")
#         o_idx, d_idx = station_name_to_idx[o_], station_name_to_idx[d_]
#         left_edges.append([o_idx, d_idx])
#         left_edges.append([d_idx, o_idx])
#     ptn.addConstr(grb.quicksum(x[i, j] for i, j in left_edges) >= 1, name = f"line_{k}")

# # 处理 Line 3 + Line 4 的情况(上海地铁)
# edges_3, edges_4 = line_routes["Line 3"], line_routes["Line 4"]
# new_edges = list(set(edges_3 + edges_4))
# left_edges = []
# for edge in new_edges:
#     o_, d_ = edge.split(",")
#     o_idx, d_idx = station_name_to_idx[o_], station_name_to_idx[d_]
#     left_edges.append([o_idx, d_idx])
#     left_edges.append([d_idx, o_idx])
# ptn.addConstr(grb.quicksum(x[i, j] for i, j in left_edges) >= 2, name = f"spec_line_3/4")

# edges_2, edges_10 = line_routes["Line 2"], line_routes["Line 10"]
# new_edges = list(set(edges_2 + edges_10))
# left_edges = []
# for edge in new_edges:
#     o_, d_ = edge.split(",")
#     o_idx, d_idx = station_name_to_idx[o_], station_name_to_idx[d_]
#     left_edges.append([o_idx, d_idx])
#     left_edges.append([d_idx, o_idx])
# ptn.addConstr(grb.quicksum(x[i, j] for i, j in left_edges) >= 2, name = f"spec_line_10/2")


# ptn.modelSense = grb.GRB.MINIMIZE
ptn.modelSense = grb.GRB.MAXIMIZE
result_routes = []
ptn.optimize()
if (ptn.status == grb.GRB.status.OPTIMAL):
    solType = 'IP_Optimal'
    ofv = ptn.getObjective().getValue()
    print(f"BEST VALUE:{ofv}")
    for i, j in x:
        if (x[i, j].x > 0.5):
            route = f"{idx_to_station_name[i]},{idx_to_station_name[j]}"
            result_routes.append(route)
            print(route)

    gap = 0
    lb = ofv
    ub = ofv
    runtime = ptn.Runtime

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M3 Pro
Thread count: 11 physical cores, 11 logical processors, using up to 11 threads

Optimize a model with 2188 rows, 2592 columns and 8088 nonzeros
Model fingerprint: 0x4bdd9904
Variable types: 0 continuous, 2592 integer (2186 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [6e+02, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective 880.0000000
Presolve time: 0.01s
Presolved: 2188 rows, 2592 columns, 8088 nonzeros
Variable types: 0 continuous, 2592 integer (2186 binary)

Root relaxation: objective 7.235985e+05, 1402 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 723598.518    0  817  880.00000 723598.518      -     -    0s
     0 

## ortools version (Open source!): Q1

In [70]:
from ortools.sat.python import cp_model
import collections
# 2. 模型初始化 (OR-Tools)
# =================================
model = cp_model.CpModel()

# 添加虚拟节点 Dummy，与Gurobi代码逻辑一致
idx_to_station_name[0] = "Dummy"
station_name_to_idx["Dummy"] = 0

N = len(station_name_to_idx.keys())
M = 2000  # "Big M" 常数，与Gurobi代码一致

# 3. 变量定义 (OR-Tools)
# =================================
x = {}
y = {}
n = {}
out_flow = collections.defaultdict(list)

# 定义 y 和 n 变量
# 索引从1开始，因为虚拟节点0没有 y 和 n 变量
for i in range(1, N):
    y[i] = model.NewBoolVar(f"y_{idx_to_station_name[i]}")
    # n[i] 的上界可以是 N 或者 M，这里用 M 保持一致
    n[i] = model.NewIntVar(0, M, f"n_{idx_to_station_name[i]}")

# 定义 x 变量，并构建出度邻接表 out_flow
# 真实站点之间的边
for station, connections in metro_network.items():
    idx_1 = station_name_to_idx[station]
    for next_station, dist in connections.items():
        idx_2 = station_name_to_idx[next_station]
        # 确保只为每个无向边创建一次变量（尽管模型处理有向边）
        if (idx_1, idx_2) not in x:
            x[idx_1, idx_2] = model.NewBoolVar(f"x_{station}_{next_station}")
        out_flow[idx_1].append(idx_2)

# 与虚拟节点相关的边
for i in range(1, N):
    x[0, i] = model.NewBoolVar(f"x_Dummy_{idx_to_station_name[i]}")
    x[i, 0] = model.NewBoolVar(f"x_{idx_to_station_name[i]}_Dummy")

# 4. 约束定义 (OR-Tools)
# =================================

# 流量守恒约束
for k in range(1, N):
    # 构建进入和流出k的完整边集，包括与虚拟节点的连接
    out_neighbors = out_flow[k] + [0]
    in_neighbors = out_flow[k] + [0] # 对于无向图的对称创建，in-neighbors等同于out-neighbors
    
    # 出度约束
    model.Add(sum(x.get((k, j), 0) for j in out_neighbors) == y[k])
    # 入度约束
    model.Add(sum(x.get((i, k), 0) for i in in_neighbors) == y[k])

# 虚拟节点流量平衡约束
model.Add(sum(x[0, i] for i in range(1, N)) == 1)
model.Add(sum(x[i, 0] for i in range(1, N)) == 1)


# 子回路消除 (MTZ) 约束
for i in range(1, N):
    # Gurobi: ptn.addConstr(n[i] <= M * y[i])
    # OR-Tools等价实现: 如果 y[i]为0, 那么 n[i]必须为0
    model.Add(n[i] == 0).OnlyEnforceIf(y[i].Not())
    # 或者直接使用线性约束，与 Gurobi 完全一致
    # model.Add(n[i] <= M * y[i]) # 这句也可以，但上面的逻辑更紧凑

# Gurobi: ptn.addConstr(n[idx_2] - n[idx_1] >= 1 + M * (x[idx_1, idx_2] - 1))
# OR-Tools 等价实现: 如果 x[idx_1, idx_2] = 1, 则 n[idx_2] >= n[idx_1] + 1
for station, connections in metro_network.items():
    idx_1 = station_name_to_idx[station]
    for next_station, dist in connections.items():
        idx_2 = station_name_to_idx[next_station]
        # 添加两个方向的约束
        model.Add(n[idx_2] >= n[idx_1] + 1).OnlyEnforceIf(x[idx_1, idx_2])

# 线路覆盖约束
for line_name, edges in line_routes.items():
    line_edge_vars = []
    for edge in edges:
        o_, d_ = edge.split(",")
        o_idx, d_idx = station_name_to_idx[o_], station_name_to_idx[d_]
        line_edge_vars.append(x[o_idx, d_idx])
        line_edge_vars.append(x[d_idx, o_idx])
    
    # Gurobi: ptn.addConstr(grb.quicksum(...) >= 1)
    model.Add(sum(line_edge_vars) >= 1)

# 特殊线路覆盖约束 (Line 3/4 和 2/10)
# 处理 Line 3 + Line 4 的情况
edges_3_4 = list(set(line_routes["Line 3"] + line_routes["Line 4"]))
line_3_4_vars = []
for edge in edges_3_4:
    o_, d_ = edge.split(",")
    o_idx, d_idx = station_name_to_idx[o_], station_name_to_idx[d_]
    line_3_4_vars.append(x[o_idx, d_idx])
    line_3_4_vars.append(x[d_idx, o_idx])
# Gurobi: ptn.addConstr(grb.quicksum(...) >= 2)
model.Add(sum(line_3_4_vars) >= 2)


# 处理 Line 2 + Line 10 的情况
edges_2_10 = list(set(line_routes["Line 2"] + line_routes["Line 10"]))
line_2_10_vars = []
for edge in edges_2_10:
    o_, d_ = edge.split(",")
    o_idx, d_idx = station_name_to_idx[o_], station_name_to_idx[d_]
    line_2_10_vars.append(x[o_idx, d_idx])
    line_2_10_vars.append(x[d_idx, o_idx])
# Gurobi: ptn.addConstr(grb.quicksum(...) >= 2)
model.Add(sum(line_2_10_vars) >= 2)


# 5. 目标函数定义 (OR-Tools)
# =================================
objective_terms = []
for station, connections in metro_network.items():
    idx_1 = station_name_to_idx[station]
    for next_station, dist in connections.items():
        idx_2 = station_name_to_idx[next_station]
        # x 变量在 Gurobi 中直接与目标系数关联
        # 在 OR-Tools 中，我们创建一个表达式
        objective_terms.append(x[idx_1, idx_2] * dist)

model.Minimize(sum(objective_terms))
# model.Maximize(sum(objective_terms))


# 6. 求解与结果输出 (OR-Tools)
# =================================
solver = cp_model.CpSolver()
# 可以设置求解时间限制，例如：solver.parameters.max_time_in_seconds = 300.0
status = solver.Solve(model)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f'BEST VALUE: {solver.ObjectiveValue()}')
    print('--- Path Found ---')
    for (i, j), var in x.items():
        if solver.Value(var) > 0:
            print(f"{idx_to_station_name[i]},{idx_to_station_name[j]}")
    print("-" * 20)
    print(f"Solver status: {solver.StatusName(status)}")
    print(f"Wall time: {solver.WallTime()}s")
else:
    print('No solution found.')

BEST VALUE: 65877.0
--- Path Found ---
莘庄,外环路
外环路,莲花路
莲花路,锦江乐园
锦江乐园,上海南站
上海南站,桂林公园
漕宝路,上海体育馆
上海体育馆,上海体育场
徐家汇,宜山路
陕西南路,上海图书馆
一大会址·黄陂南路,陕西南路
虹桥火车站,国家会展中心L17
虹桥2号航站楼,虹桥火车站
陆家嘴,豫园
浦东南路,陆家嘴
世纪大道,浦东南路
龙阳路,迎春路
宜山路,虹桥路
虹桥路,宋园路
东安路,龙华中路
上海体育场,东安路
春申路,莘庄
民生路,源深体育中心
源深体育中心,世纪大道
东明路,华鹏路
龙华中路,后滩
后滩,长清路
长清路,耀华路
耀华路,成山路
大世界,一大会址·黄陂南路
成山路,东明路
杨高中路,民生路
龙溪路,上海动物园
虹桥1号航站楼,虹桥2号航站楼
上海动物园,虹桥1号航站楼
水城路,龙溪路
伊犁路,水城路
宋园路,伊犁路
交通大学,徐家汇
上海图书馆,交通大学
豫园,大世界
桂林公园,漕宝路
华鹏路,下南路
下南路,北蔡
北蔡,陈春路
陈春路,莲溪路
莲溪路,华夏中路
华夏中路,龙阳路
迎春路,杨高中路
Dummy,春申路
国家会展中心L17,Dummy
--------------------
Solver status: OPTIMAL
Wall time: 146.610394s


## ortools for Q2

In [72]:
from ortools.sat.python import cp_model
import collections
# 2. 模型初始化 (OR-Tools)
# =================================
model = cp_model.CpModel()

# 添加虚拟节点 Dummy，与Gurobi代码逻辑一致
idx_to_station_name[0] = "Dummy"
station_name_to_idx["Dummy"] = 0

N = len(station_name_to_idx.keys())
M = 2000  # "Big M" 常数，与Gurobi代码一致

# 3. 变量定义 (OR-Tools)
# =================================
x = {}
y = {}
n = {}
out_flow = collections.defaultdict(list)

# 定义 y 和 n 变量
# 索引从1开始，因为虚拟节点0没有 y 和 n 变量
for i in range(1, N):
    y[i] = model.NewBoolVar(f"y_{idx_to_station_name[i]}")
    # n[i] 的上界可以是 N 或者 M，这里用 M 保持一致
    n[i] = model.NewIntVar(0, M, f"n_{idx_to_station_name[i]}")

# 定义 x 变量，并构建出度邻接表 out_flow
# 真实站点之间的边
for station, connections in metro_network.items():
    idx_1 = station_name_to_idx[station]
    for next_station, dist in connections.items():
        idx_2 = station_name_to_idx[next_station]
        # 确保只为每个无向边创建一次变量（尽管模型处理有向边）
        if (idx_1, idx_2) not in x:
            x[idx_1, idx_2] = model.NewBoolVar(f"x_{station}_{next_station}")
        out_flow[idx_1].append(idx_2)

# 与虚拟节点相关的边
for i in range(1, N):
    x[0, i] = model.NewBoolVar(f"x_Dummy_{idx_to_station_name[i]}")
    x[i, 0] = model.NewBoolVar(f"x_{idx_to_station_name[i]}_Dummy")

# 4. 约束定义 (OR-Tools)
# =================================

# 流量守恒约束
for k in range(1, N):
    # 构建进入和流出k的完整边集，包括与虚拟节点的连接
    out_neighbors = out_flow[k] + [0]
    in_neighbors = out_flow[k] + [0] # 对于无向图的对称创建，in-neighbors等同于out-neighbors
    
    # 出度约束
    model.Add(sum(x.get((k, j), 0) for j in out_neighbors) == y[k])
    # 入度约束
    model.Add(sum(x.get((i, k), 0) for i in in_neighbors) == y[k])

# 虚拟节点流量平衡约束
model.Add(sum(x[0, i] for i in range(1, N)) == 1)
model.Add(sum(x[i, 0] for i in range(1, N)) == 1)


# 子回路消除 (MTZ) 约束
for i in range(1, N):
    # Gurobi: ptn.addConstr(n[i] <= M * y[i])
    # OR-Tools等价实现: 如果 y[i]为0, 那么 n[i]必须为0
    model.Add(n[i] == 0).OnlyEnforceIf(y[i].Not())
    # 或者直接使用线性约束，与 Gurobi 完全一致
    # model.Add(n[i] <= M * y[i]) # 这句也可以，但上面的逻辑更紧凑

# Gurobi: ptn.addConstr(n[idx_2] - n[idx_1] >= 1 + M * (x[idx_1, idx_2] - 1))
# OR-Tools 等价实现: 如果 x[idx_1, idx_2] = 1, 则 n[idx_2] >= n[idx_1] + 1
for station, connections in metro_network.items():
    idx_1 = station_name_to_idx[station]
    for next_station, dist in connections.items():
        idx_2 = station_name_to_idx[next_station]
        # 添加两个方向的约束
        model.Add(n[idx_2] >= n[idx_1] + 1).OnlyEnforceIf(x[idx_1, idx_2])

# 线路覆盖约束
# for line_name, edges in line_routes.items():
#     line_edge_vars = []
#     for edge in edges:
#         o_, d_ = edge.split(",")
#         o_idx, d_idx = station_name_to_idx[o_], station_name_to_idx[d_]
#         line_edge_vars.append(x[o_idx, d_idx])
#         line_edge_vars.append(x[d_idx, o_idx])
    
#     # Gurobi: ptn.addConstr(grb.quicksum(...) >= 1)
#     model.Add(sum(line_edge_vars) >= 1)

# # 特殊线路覆盖约束 (Line 3/4 和 2/10)
# # 处理 Line 3 + Line 4 的情况
# edges_3_4 = list(set(line_routes["Line 3"] + line_routes["Line 4"]))
# line_3_4_vars = []
# for edge in edges_3_4:
#     o_, d_ = edge.split(",")
#     o_idx, d_idx = station_name_to_idx[o_], station_name_to_idx[d_]
#     line_3_4_vars.append(x[o_idx, d_idx])
#     line_3_4_vars.append(x[d_idx, o_idx])
# # Gurobi: ptn.addConstr(grb.quicksum(...) >= 2)
# model.Add(sum(line_3_4_vars) >= 2)


# # 处理 Line 2 + Line 10 的情况
# edges_2_10 = list(set(line_routes["Line 2"] + line_routes["Line 10"]))
# line_2_10_vars = []
# for edge in edges_2_10:
#     o_, d_ = edge.split(",")
#     o_idx, d_idx = station_name_to_idx[o_], station_name_to_idx[d_]
#     line_2_10_vars.append(x[o_idx, d_idx])
#     line_2_10_vars.append(x[d_idx, o_idx])
# # Gurobi: ptn.addConstr(grb.quicksum(...) >= 2)
# model.Add(sum(line_2_10_vars) >= 2)


# 5. 目标函数定义 (OR-Tools)
# =================================
objective_terms = []
for station, connections in metro_network.items():
    idx_1 = station_name_to_idx[station]
    for next_station, dist in connections.items():
        idx_2 = station_name_to_idx[next_station]
        # x 变量在 Gurobi 中直接与目标系数关联
        # 在 OR-Tools 中，我们创建一个表达式
        objective_terms.append(x[idx_1, idx_2] * dist)

# model.Minimize(sum(objective_terms))
model.Maximize(sum(objective_terms))


# 6. 求解与结果输出 (OR-Tools)
# =================================
solver = cp_model.CpSolver()
# 可以设置求解时间限制，例如：solver.parameters.max_time_in_seconds = 300.0
status = solver.Solve(model)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f'BEST VALUE: {solver.ObjectiveValue()}')
    print('--- Path Found ---')
    for (i, j), var in x.items():
        if solver.Value(var) > 0:
            print(f"{idx_to_station_name[i]},{idx_to_station_name[j]}")
    print("-" * 20)
    print(f"Solver status: {solver.StatusName(status)}")
    print(f"Wall time: {solver.WallTime()}s")
else:
    print('No solution found.')

BEST VALUE: 354010.0
--- Path Found ---
上海南站,石龙路
漕宝路,桂林公园
上海体育馆,漕宝路
徐家汇,衡山路
衡山路,常熟路
常熟路,肇嘉浜路
陕西南路,一大会址·新天地
一大会址·黄陂南路,人民广场
人民广场,南京东路
汉中路,江宁路
上海火车站,中潭路
虹桥2号航站楼,虹桥1号航站楼
淞虹路,虹桥2号航站楼
北新泾,淞虹路
威宁路,北新泾
娄山关路,威宁路
中山公园,江苏路
江苏路,交通大学
静安寺,一大会址·黄陂南路
南京西路,汉中路
南京东路,陆家嘴
陆家嘴,豫园
浦东南路,浦东大道
世纪大道,浦东南路
龙阳路,芳华路
石龙路,龙漕路
龙漕路,漕溪路
漕溪路,宜山路
宜山路,桂林路
虹桥路,延安西路
延安西路,中山公园
金沙江路,大渡河路
曹杨路,武宁
镇坪路,岚皋路
中潭路,镇坪路
宝山路,上海火车站
虹口足球场,赤峰路
赤峰路,大柏树
大柏树,江湾镇
江湾镇,殷高西路
殷高西路,长江南路
长江南路,殷高路
海伦路,宝山路
大连路,提篮桥
杨树浦路,大连路
浦东大道,杨树浦路
蓝村路,塘桥
塘桥,南浦大桥
南浦大桥,西藏南路
西藏南路,中华艺术宫
大木桥路,东安路
东安路,上海体育场
上海体育场,上海体育馆
港城路,外高桥保税区北
外高桥保税区北,航津路
航津路,外高桥保税区南
外高桥保税区南,洲海路
洲海路,五洲大道
五洲大道,东靖路
东靖路,巨峰路
巨峰路,杨高北路
云山路,龙居路
德平路,云山路
北洋泾路,德平路
民生路,北洋泾路
上海儿童医学中心,蓝村路
临沂新村,上海儿童医学中心
高科西路,临沂新村
东明路,华鹏路
东方体育中心,龙耀路
顾村公园,锦秋路
祁华路,顾村公园
上海大学,祁华路
南陈路,上海大学
上大路,南陈路
场中路,上大路
大场镇,场中路
行知路,大场镇
大华三路,行知路
新村路,大华三路
岚皋路,新村路
长寿路,武宁路
肇嘉浜路,嘉善路
龙华中路,大木桥路
后滩,龙华中路
长清路,后滩
耀华路,成山路
杨高南路,高科西路
锦绣路,杨高南路
芳华路,锦绣路
江浦路,鞍山新村
鞍山新村,四平路
四平路,邮电新村
西藏北路,虹口足球场
中兴路,西藏北路
曲阜路,中兴路
大世界,老西门
老西门,陆家浜路
陆家浜路,小南门
中华艺术宫,耀华路
成山路,东明路
桂林路,吴中路
嘉善路,打浦桥
打浦桥

## Reconstruct from result

In [64]:
station_line_map = collections.defaultdict(set)
for line_n, line_arr in line_routes.items():
    for edge_n in line_arr:
        o_, d_ = edge_n.split(",") 
        station_line_map[f"{o_},{d_}"].add(line_n)
        station_line_map[f"{d_},{o_}"].add(line_n)
print(station_line_map['宜山路,虹桥路'])
print(station_line_map['上海体育场,上海体育馆'])
print(station_line_map['莘庄,外环路'])
print(station_line_map['外环路,莘庄'])

def order_path_segments(unordered_segments):
    """
    将一个无序的路径段列表整理成一个从 "Dummy" 出发并返回到 "Dummy" 的有序路径。

    Args:
        unordered_segments (list): 一个字符串列表，每个字符串格式为 "起点,终点"。
                                   例如: ["奉贤新城,Dummy", "Dummy,西岑", ...]。

    Returns:
        list: 一个有序的路径段列表，格式与输入相同。
    """
    # 步骤 1: 创建一个从起点到终点的快速查找字典
    # 这比每次都在列表中搜索起点要高效得多
    path_map = {}
    for segment in unordered_segments:
        try:
            # 去除可能存在的空格并按逗号分割
            start_node, end_node = [s.strip() for s in segment.split(',')]
            path_map[start_node] = end_node
        except ValueError:
            print(f"警告: 忽略格式错误的行: '{segment}'")
            continue
            
    # 检查数据是否完整，路径是否能从Dummy开始
    if "Dummy" not in path_map:
        raise ValueError("路径错误: 在数据中找不到 'Dummy' 作为起点。")

    # 步骤 2: 从 "Dummy" 开始，一步步重构路径
    ordered_path_segments = []
    
    # 初始化路径的起点
    current_node = "Dummy"
    
    # 循环，直到我们完成整个环路
    # 我们期望环路的长度等于原始段的数量
    for _ in range(len(path_map)):
        # 找到当前节点的下一站
        # 如果current_node不在字典的键中，说明路径断了，这不应该发生
        if current_node not in path_map:
             raise ValueError(f"路径断裂: 找不到从 '{current_node}' 出发的路段。")
        
        next_node = path_map[current_node]
        
        # 将找到的有序路段添加到结果列表中
        ordered_path_segments.append(f"{current_node},{next_node}")
        
        # 更新当前节点，为下一次迭代做准备
        current_node = next_node
        
        # 如果我们已经回到了起点Dummy，说明环路已经完成，可以提前结束
        if current_node == "Dummy":
            break
            
    # 步骤 3: 验证路径是否完整
    if len(ordered_path_segments) != len(path_map):
        print("警告: 生成的有序路径长度与原始数据不匹配，可能存在孤立的环或断路。")

    return ordered_path_segments


# 调用函数进行排序
try:
    ordered_path = order_path_segments(result_routes)
    
    print("整理后的有序路径:")
    for segment in ordered_path:
        if "Dummy" in segment :
            continue 
        print(segment,station_line_map[segment])
except (ValueError, KeyError) as e:
    print(f"处理出错: {e}")

{'Line 4', 'Line 3'}
{'Line 4'}
{'Line 1'}
{'Line 1'}
整理后的有序路径:
滴水湖,临港大道 {'Line 16'}
临港大道,书院 {'Line 16'}
书院,惠南东 {'Line 16'}
惠南东,惠南 {'Line 16'}
惠南,野生动物园 {'Line 16'}
野生动物园,新场 {'Line 16'}
新场,航头东 {'Line 16'}
航头东,鹤沙航城 {'Line 16'}
鹤沙航城,周浦东 {'Line 16'}
周浦东,罗山路 {'Line 16'}
罗山路,华夏中路 {'Line 16'}
华夏中路,龙阳路 {'Line 16'}
龙阳路,芳华路 {'Line 7'}
芳华路,锦绣路 {'Line 7'}
锦绣路,杨高南路 {'Line 7'}
杨高南路,高科西路 {'Line 7'}
高科西路,临沂新村 {'Line 6'}
临沂新村,上海儿童医学中心 {'Line 6'}
上海儿童医学中心,蓝村路 {'Line 6'}
蓝村路,塘桥 {'Line 4'}
塘桥,南浦大桥 {'Line 4'}
南浦大桥,西藏南路 {'Line 4'}
西藏南路,中华艺术宫 {'Line 8'}
中华艺术宫,耀华路 {'Line 8'}
耀华路,成山路 {'Line 8'}
成山路,东明路 {'Line 13'}
东明路,华鹏路 {'Line 13'}
华鹏路,下南路 {'Line 13'}
下南路,北蔡 {'Line 13'}
北蔡,陈春路 {'Line 13'}
陈春路,莲溪路 {'Line 13'}
莲溪路,御桥 {'Line 18'}
御桥,康恒路 {'Line 11'}
康恒路,浦三路 {'Line 11'}
浦三路,三林东 {'Line 11'}
三林东,三林 {'Line 11'}
三林,东方体育中心 {'Line 11'}
东方体育中心,龙耀路 {'Line 11'}
龙耀路,云锦路 {'Line 11'}
云锦路,龙华 {'Line 11'}
龙华,上海游泳馆 {'Line 11'}
上海游泳馆,徐家汇 {'Line 11'}
徐家汇,衡山路 {'Line 1'}
衡山路,常熟路 {'Line 1'}
常熟路,肇嘉浜路 {'Line 7'}
肇嘉浜路,嘉善路 {'Line 9'}
嘉善路

In [ ]:
# 【所需边数最小的（44条边）】
# 国家会展中心L17,虹桥火车站,2523
# 虹桥火车站,虹桥2号航站楼,696
# 虹桥2号航站楼,淞虹路,5921
# 淞虹路,北新泾,1367
# 北新泾,威宁路,1260
# 威宁路,娄山关路,1661
# 娄山关路,长风公园,1487
# 长风公园,大渡河路,784
# 大渡河路,金沙江路,1676
# 金沙江路,曹杨路,903
# 曹杨路,武宁,1050
# 武宁,武定路,970
# 武定路,静安寺,1030
# 静安寺,常熟路,1091
# 常熟路,陕西南路,939
# 陕西南路,一大会址·新天地,1591
# 一大会址·新天地,老西门,880
# 老西门,陆家浜路,825
# 陆家浜路,小南门,1475
# 小南门,商城路,2398
# 商城路,世纪大道,979
# 世纪大道,源深体育中心,890
# 源深体育中心,民生路,907
# 民生路,杨高中路,928
# 杨高中路,迎春路,633
# 迎春路,龙阳路,2275
# 龙阳路,华夏中路,4355
# 华夏中路,莲溪路,1589
# 莲溪路,御桥,1342
# 御桥,浦三路,3221
# 浦三路,三林东,1613
# 三林东,三林,1126
# 三林,东方体育中心,3516
# 东方体育中心,龙耀路,2164
# 龙耀路,云锦路,934
# 云锦路,龙华,945
# 龙华,龙漕路,1124
# 龙漕路,石龙路,1484
# 石龙路,上海南站,1314
# 上海南站,锦江乐园,2089
# 锦江乐园,莲花路,1633
# 莲花路,外环路,1459
# 外环路,莘庄,1303
# 莘庄,春申路,1588
# 69938 



| 序号 | 线路 | 起点 | 终点| 距离 | 
|:---:| :---: |:----: | :---:| :---: |
| 1 | L17 | 国家会展中心L17 | 虹桥火车站 | 2523 |
| 2 | L2 | 虹桥火车站 | 虹桥2号航站楼 | 696 | 
| 3 | L10 | 虹桥2号航站楼 | 虹桥1号航站楼 | 2026 |
|4 | L10  | 虹桥1号航站楼 | 上海动物园 | 2016 |
| 5 | L10 | 上海动物园 | 龙溪路 | 1244| 
|6 | L10| 龙溪路 | 水城路 | 1303 |
| 7| L10| 水城路 | 伊犁路 | 1167 |
| 8| L10 | 伊犁路 | 宋园路 | 800 |
| 9| L10| 宋园路 | 虹桥路 | 1053 |
| 10 | L3 | 虹桥路 | 宜山路 | 1323 |
| 11| L9 | 宜山路 | 徐家汇 | 1505 |
| 12| L11 | 徐家汇 | 交通大学 | 905 |
| 13| L11 | 交通大学 | 上海图书馆 | 1145 |
| 14| L11 | 上海图书馆 | 陕西南路 | 1618 |
| 15 | L1 | 陕西南路 | 一大会址·黄陂南路 | 1407 |
|16 |  L14 | 一大会址·黄陂南路 | 大世界 | 710 | 
| 17| L14 | 大世界 | 豫园 | 740 |
|8| L14 | 豫园 | 陆家嘴 | 1600 |
|9| L14 | 陆家嘴 | 浦东南路 | 1010 |
|20| L2 | 浦东南路 | 世纪大道 | 1245 |
|21| L6 | 世纪大道 | 源深体育中心 | 890 |
|22| L6| 源深体育中心 | 民生路 | 907 |
|23| L18 | 民生路 | 杨高中路 | 928 |
|24| L18| 杨高中路 | 迎春路 | 633 |
|25| L18 | 迎春路 | 龙阳路 | 2275 |
|26| L16 | 龙阳路 | 华夏中路 | 4355 |
|27| L13 | 华夏中路 | 莲溪路 | 1589 |
|28| L13| 莲溪路 | 陈春路 | 1239 |
|29 |L13 | 陈春路 | 北蔡 | 980 |
|30 | L13| 北蔡 | 下南路 | 1128 |
|31|L13 | 下南路 | 华鹏路 | 1361 |
|32| L13| 华鹏路 | 东明路 | 1464 |
|33 | L13| 东明路 | 成山路 | 1397 |
|34 |L8 | 成山路 | 耀华路 | 908 |
|35| L7 | 耀华路 | 长清路 | 926 |
|36| L7| 长清路 | 后滩 | 1263 |
|37| L7| 后滩 | 龙华中路 | 2247 |
|38| L7| 龙华中路 | 东安路 | 728 |
|39| L4| 东安路 | 上海体育场 | 1220 |
|40 | L4| 上海体育场 | 上海体育馆 | 685 |
|41 |L1 | 上海体育馆 | 漕宝路 | 1534 |
|42 |L12 | 漕宝路 | 桂林公园 | 1284 |
|43 |L15 | 桂林公园 | 上海南站 | 1828 |
|44 |L1 | 上海南站 | 锦江乐园 | 2089 |
|45 |L1 | 锦江乐园 | 莲花路 | 1633 |
|46 |L1 | 莲花路 | 外环路 | 1459 |
|47 |L1 | 外环路 | 莘庄 | 1303 |
|48 |L5 | 莘庄 | 春申路 | 1588 |
